# Prerequisites

In [2]:
import json
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import transformers

# package in . directory
from bichoice import utils
from bichoice.data_processor import (
    C3BinaryExample,
    C3BinaryDataProcessor,
)

In [24]:
assert transformers.__version__ == '4.1.1'

# declare a namespace
D = utils.GlobalSettings({
        'DATADIR': './data/',
        'OUTDIR': './outputs/csv-data/',
        'MODELDIR': './outputs/bichoice_output/checkpoint-3/',
    })

if not os.path.exists(D.OUTDIR):
    os.makedirs(D.OUTDIR)

# Reshape C3 as Binary Classification

In [5]:
def C3Example_to_C3BinaryExample(eList):
    '''
    create `C3BinaryExample`s from `C3Example`s.
    
    Args
    ----
    `eList` : a list of `bichoice.utils.C3Example`
    
    Return
    ------
    `out` : a list of `bichioce.data_processor.C3BinaryExample`.
    '''
    out = []
    for e in eList:
        passage = ''.join(e.sentences)
        question = e.question
        answer = e.options[e.label]
        for o in e.options:
            if o == answer:
                continue
            out.append(C3BinaryExample(passage, question, answer, o, 0))
            out.append(C3BinaryExample(passage, question, o, answer, 1))
    return out

In [6]:
def C3BinaryExamples_to_dataframe(eList):
    '''
    convert a list of `C3BinaryExample` to `pandas.DataFrame`
    
    Args
    ----
    `eList` : a list of `C3BinaryExample`
    
    Return
    ------
    `out` : `pandas.DataFrame` 
    '''
    out = {
        'passage': [e.passage for e in eList],
        'question': [e.question for e in eList],
        'choice_0': [e.choice_0 for e in eList],
        'choice_1': [e.choice_1 for e in eList],
        'label': [e.label for e in eList],
    }
    return pd.DataFrame(out)

In [7]:
train = utils.get_all_C3examples(os.path.join(D.DATADIR, 'c3-m-train.json'))
train+= utils.get_all_C3examples(os.path.join(D.DATADIR, 'c3-d-train.json'))

valid = utils.get_all_C3examples(os.path.join(D.DATADIR, 'c3-m-dev.json'))
valid+= utils.get_all_C3examples(os.path.join(D.DATADIR, 'c3-d-dev.json'))

test = utils.get_all_C3examples(os.path.join(D.DATADIR, 'c3-m-test.json'))
test+= utils.get_all_C3examples(os.path.join(D.DATADIR, 'c3-d-test.json'))

In [8]:
bi_train_df = C3BinaryExamples_to_dataframe(C3Example_to_C3BinaryExample(train))
bi_valid_df = C3BinaryExamples_to_dataframe(C3Example_to_C3BinaryExample(valid))
bi_test_df = C3BinaryExamples_to_dataframe(C3Example_to_C3BinaryExample(test))

In [14]:
print('-----BEFORE RESHAPING-----')
print('#examples of train', len(train))
print('#examples of validation', len(valid))
print('#examples of test', len(test))

print('-----AFTER RESHAPING-----')
print('#examples of train', len(bi_train_df))
print('#examples of validation', len(bi_valid_df))
print('#examples of test', len(bi_test_df))

-----BEFORE RESHAPING-----
#examples of train 11869
#examples of validation 3816
#examples of test 3892
-----AFTER RESHAPING-----
#examples of train 66192
#examples of validation 21286
#examples of test 21626


In [18]:
def show_c3_binary_example(e):
    '''show all info of a single `C3BinaryExample`'''
    print('-----PASSAGE-----')
    print(e.passage)
    print('-----QUESTION-----')
    print(e.question)
    print('-----CHOICE_0-----')
    print(e.choice_0)
    print('-----CHOICE_1-----')
    print(e.choice_1)
    print('-----LABEL-----')
    print(e.label)

In [21]:
show_c3_binary_example(bi_train_df.loc[random.choice(bi_train_df.index)])

-----PASSAGE-----
2010年12月升空的美国的“发现号”航天飞机里面除了6名航天员外，还有一位与航天员十分相似的机器人——“机器人航天员2”。虽然一些简单的老式机器人，如月球、火星的探测器早已参加到太空探索的行列中，并且依然是太空探测的主力军，但像“机器人航天员2”这种人形的机器人的升空则是人类的第一次。
“机器人航天员2”昵称R2，它的外形十分像人，有头、臂和躯干，是美国国家航空航天局和通用汽车公司联合研制的仿真机器人，造价250万美元。它集各种先进控制、传感和视觉等技术于一身，并且其接口不需要做任何调整就可以与空间站上的各项设施连接。它可以使用所有航天员使用的工具，协助太空人处理一些枯燥或危险的作业，节省太空人的时间和保证太空人的安全。目前R2只能在实验室中工作，未来提升功能后，将能在空间站内四处移动和进行太空行走。
尽管机器人航天员的面世令人感到欣喜，但与人类航天员相比，其能力还是十分有限。因此，机器人航天员不是替代人类，而是作为航天员的出色“同伴”，辅助人类一起承载航天任务中的重要职责。
-----QUESTION-----
根据文章我们可以知道：
-----CHOICE_0-----
R2将会辅助人类做好航天任务中的重要工作
-----CHOICE_1-----
R2即将取代老式机器人成为太空探测的主力军
-----LABEL-----
0


In [23]:
# save reshaped dataset
bi_train_df.to_csv(os.path.join(D.OUTDIR, 'binary-train.csv'),
                    index=False, encoding='utf-8')
bi_valid_df.to_csv(os.path.join(D.OUTDIR, 'binary-dev.csv'),
                    index=False, encoding='utf-8')
bi_train_df.to_csv(os.path.join(D.OUTDIR, 'binary-test.csv'),
                    index=False, encoding='utf-8')

# Load Fine-tuned Model

In [ ]:
if not os.path.exists()